# import

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set dataset

In [3]:
data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token.txt"
pairs = []
with open(data_path,"r",encoding="utf-8") as f:
    for line in f:
        pair = line.split("\t")
        pairs.append(pair)

def get_pair(data):
    
